In [5]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=7750709688909cc39d19998c4a408155745a5bf425a2a20fa23aeb8bacc25674
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [11]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

# Ruta a la carpeta con las imágenes
folder_path = '/content/fotos'

# Obtener la lista de archivos de imagen en la carpeta
image_files = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Función para convertir imagen a escala de grises
def convert_to_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Función para segmentar la imagen basada en umbrales
def segment_image(image):
    gray_image = convert_to_grayscale(image)
    # Aplicar un umbral global
    _, thresholded = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY)
    return thresholded

# Procesar y mostrar las imágenes
for image_file in image_files:
    image_path = os.path.join(folder_path, image_file)
    image = cv2.imread(image_path)

    if image is None:
        continue

    segmented_image = segment_image(image)

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title('Imagen Original')

    plt.subplot(1, 2, 2)
    plt.imshow(segmented_image, cmap='gray')
    plt.title('Imagen Segmentada')

    plt.show()


Output hidden; open in https://colab.research.google.com to view.

In [12]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image

# Ruta a la carpeta con las imágenes
folder_path = '/content/fotos'  # Cambia esto a la ruta correcta de tu carpeta

# Obtener la lista de archivos de imagen en la carpeta
image_files = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Cargar el modelo VGG16 preentrenado sin la última capa de clasificación
base_model = VGG16(weights='imagenet', include_top=False)
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block5_pool').output)

# Función para preparar la imagen para la CNN
def prepare_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array)

# Procesar y extraer características de las imágenes
features = []

for image_file in image_files:
    image_path = os.path.join(folder_path, image_file)
    img = prepare_image(image_path)
    features.append(model.predict(img).flatten())

# Convertir la lista de características en una matriz
feature_matrix = np.array(features)
print("Matriz de características:", feature_matrix.shape)

# Mostrar las imágenes y sus características extraídas
for i, image_file in enumerate(image_files):
    image_path = os.path.join(folder_path, image_file)
    img = cv2.imread(image_path)
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title('Imagen Original')

    plt.subplot(1, 2, 2)
    plt.bar(range(len(features[i])), features[i])
    plt.title('Características Extraídas')

    plt.show()


Output hidden; open in https://colab.research.google.com to view.